In [1]:
import time
import torch
from torch import nn,optim
from presentation import *
import torchvision

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class LENet(nn.Module):
    def __init__(self):
        super(LENet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4,120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    
    def forward(self, img):
        feature = self.conv(img)
        return self.fc(feature.view(img.shape[0], -1))

net = LENet()
print(net)
                                  

LENet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [3]:
# 下载数据集 torchvision.datasets：加载数据的函数和常用的的数据及接口
mnist_train = torchvision.datasets.FashionMNIST(root='D:\Repos\PycharmProjects\Datasets\FashionMNIST',
                                                train=True, download=True,
                                                transform=transforms.ToTensor())  # 将所有数据转换成Tensor (C,H,W)
mnist_test = torchvision.datasets.FashionMNIST(root='D:\Repos\PycharmProjects\Datasets\FashionMNIST',
                                               train=False, download=True, transform=transforms.ToTensor())
# print(len(mnist_train),len(mnist_test))

# feature, lable = mnist_train[20]
# print(feature,lable)

In [4]:
batch_size = 256
num_epochs = 10

train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

loss = nn.CrossEntropyLoss()

lr = 0.001
optimizer = torch.optim.Adam(net.parameters(),lr=lr)



In [5]:
def evaluate_accuracy(test_iter, net, device):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in test_iter:
            net.eval()
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train()
            n += y.shape[0]
    return acc_sum / n
        

In [6]:
def train(net, train_iter, test_iter, optimizer, loss, device, num_epochs):
    # 设置GPU训练
    net = net.to(device)
    print("training on ",device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            # 设置GPU训练
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net, device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'% (epoch+1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time()-start))
            

In [7]:
train(net, train_iter, test_iter, optimizer, loss, device, num_epochs)

training on  cuda
epoch 1, loss 1.8206, train acc 0.329, test acc 0.562, time 9.6 sec
epoch 2, loss 0.4867, train acc 0.612, test acc 0.650, time 6.9 sec
epoch 3, loss 0.2683, train acc 0.695, test acc 0.716, time 7.2 sec
epoch 4, loss 0.1766, train acc 0.736, test acc 0.739, time 7.1 sec
epoch 5, loss 0.1299, train acc 0.754, test acc 0.753, time 6.9 sec
epoch 6, loss 0.1008, train acc 0.766, test acc 0.766, time 7.0 sec
epoch 7, loss 0.0811, train acc 0.779, test acc 0.780, time 6.8 sec
epoch 8, loss 0.0675, train acc 0.791, test acc 0.783, time 7.0 sec
epoch 9, loss 0.0575, train acc 0.800, test acc 0.799, time 7.3 sec
epoch 10, loss 0.0495, train acc 0.811, test acc 0.798, time 7.0 sec
